In [ ]:
# (1)         (2)         (3)
# RDBMS       ORM-CORE    ORM
#             Meta
#        engine       declrative-base 
#     connect dialect
#         session
# 물리적 Table   Table 객체 Class 객체
# Database 저장   meta.tables base 저장
#                 저장    
# -----------------------------------
#                     Mapper: namespace

# 해결책: declrative-base를 계속 날리고 생성하면 된다

In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy.orm.session import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.schema import Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.orm import relationship

In [ ]:
sess.close()
engine.connect().close()
engine.dispose()


In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker(engine)
sess = session()

In [ ]:
base = declarative_base()

In [ ]:
base.metadata.reflect(engine)

In [ ]:
base.metadata.tables

In [ ]:
class Artist(base):
    __table__ = base.metadata.tables['ARTIST']

In [ ]:
sing = sess.query(Artist).all()[0]

In [ ]:
class Album(base):
    __table__ = base.metadata.tables['ALBUM']

In [ ]:
b = sess.query(Album).filter(Album.PK == 1).one()

In [ ]:
del base

In [ ]:
class Artist(base):
    __tablename__ = 'T_ARTIST'
    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text)

class Album(base):
    __tablename__ = 'T_ALBUM'
    pk = Column('PK', Integer, primary_key = True)
    name = Column('NAME', Text)
    fk = Column('FK', ForeignKey(Artist.pk))

In [ ]:
base.metadata.create_all(engine)

In [ ]:
a = Artist(name='수지')
sess.add(a)
sess.commit()

sess.add(Album(name='수지앨범', fk = a.pk))
sess.commit()

In [ ]:
sess.query(Artist).join(Album).all()

In [ ]:
del base    # base 없앰
base = declarative_base()   # base 새로 만들고
base.metadata.reflect(engine)   # engine으로부터 reflect

In [ ]:
class Artist(base):
    __table__ = base.metadata.tables['T_ARTIST']

    pk = base.metadata.tables['T_ARTIST'].c.PK
    name = base.metadata.tables['T_ARTIST'].c.NAME
    albums = relationship('Album', back_populates='artist', uselist=True)

class Album(base):
    __table__ = base.metadata.tables['T_ALBUM']
    
    pk = base.metadata.tables['T_ALBUM'].c.PK
    name = base.metadata.tables['T_ALBUM'].c.NAME
    fk = base.metadata.tables['T_ALBUM'].c.FK
    artist = relationship('Artist', back_populates='albums', uselist=False)

In [ ]:
a = sess.query(Artist).all()[0]

In [ ]:
a.albums.append(Album(name='수지앨범2', fk=a.pk))
# 객체 지향에 맞지 않는 방식

In [ ]:
sess.dirty, sess.is_modified(a)

In [ ]:
sess.commit()

In [ ]:
len(a.albums), a.albums[-1].name

In [ ]:
del base
base = declarative_base()
base.metadata.reflect(engine)

In [ ]:
class Artist(base):
    __table__ = base.metadata.tables['T_ARTIST']

    pk = base.metadata.tables['T_ARTIST'].c.PK
    name = base.metadata.tables['T_ARTIST'].c.NAME
    albums = relationship('Album', back_populates='artist', uselist=True)

    def addAlbum(self, s, name):
        s.add(Album(name=name, fk=self.pk))
        s.commit()
    def delAlbum(self, s, name):
        # album = s.query(Album).filter(Album.name == name).one()
        # 혹은 이런식으로
        album = list(filter(lambda a: a.name == name, self.albums))[0]
        self.albums.remove(album)
        s.commit()

class Album(base):
    __table__ = base.metadata.tables['T_ALBUM']
    
    pk = base.metadata.tables['T_ALBUM'].c.PK
    name = base.metadata.tables['T_ALBUM'].c.NAME
    fk = base.metadata.tables['T_ALBUM'].c.FK
    artist = relationship('Artist', back_populates='albums', uselist=False)

In [ ]:
a = sess.query(Artist).all()[0]

In [ ]:
a.addAlbum(sess, '수지앨범3')

In [ ]:
len(a.albums), a.albums[-1].name

In [ ]:
a.delAlbum(sess, '수지앨범3')

In [ ]:
len(a.albums), a.albums[-1].name

In [ ]:
# 숙제
# 포스팅 + 해시태그

In [ ]:
# -------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------

In [ ]:
# Scraping => 사이트마다의 특성을 알아야함
# 특히, 한국말로 된 한국 사이트 특징을 알아야함

In [ ]:
# 뽐뿌에서 게시글 긁고 댓글 긁는거 해보자
# pagenation - page 숫자 / 눌러보고 주소가 바뀌는지 확인하는게 좋다

In [ ]:
from requests import request

url = 'https://www.ppomppu.co.kr/zboard/zboard.php'
params = {
    'id':'freeboard',
    'page': 1
}

resp = request('GET', url, params=params)

In [ ]:
resp.headers['content-type']

In [ ]:
resp.encoding

In [ ]:
resp.text
# text에서 한글나온다고 잘 나온느게 아님, 코드값이 다름

In [ ]:
from bs4 import BeautifulSoup

dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
from requests.compat import urljoin

for tag in (dom.select('#revolution_main_table tr#headNotice ~ tr > td:nth-child(3) > a')):
    print(urljoin(url, tag.attrs['href']))

In [ ]:
resp = request('GET', 'https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1623&no=8711417')
# 본문 가져오기

In [ ]:
resp.headers['content-type']

In [ ]:
dom = BeautifulSoup(resp.text, 'html.parser')
# parser 마다 결과가 다른데 찾을 수 있거나 없을 때가 있음
# 왜냐하면 겹치는게 존재, class가 중복일 수도 있음
# class 중복일 때 parser마다 해석하는 방식이 다르기 때문이다

In [ ]:
for tag in dom.select('table table table > tr:first-child > :first-child'):
    print(tag)
# parser에 상관없이 동작함

In [ ]:
url = 'https://pythonscraping.com/pages/javascript/ajaxDemo.html'
resp = request('GET', url)

In [ ]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
# javascript, jquery 등등 다 다르긴 한데 해석해보자
import re

a = re.search(r'^\s*url\s*:\s*[\'\"]?(.+?)[\'\"]?,?$', dom.body.script.text, re.MULTILINE).group(1)
# 이러면 이게 url을 리턴한다

In [ ]:
url = urljoin(resp.request.url, a)
resp = request('GET', url)
resp.text

In [ ]:
url = 'https://brunch.co.kr/search?q=수지'
resp = request('GET', url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
dom.find_all(string=re.compile('수지'))
# 하나만 뜸 따라서 xhr을 확인해야함
# 페이지 전체가 바뀌지 않고 content가 바뀌었기 때문이다

In [ ]:
# 파라미터를 받아오기 위해 form을 찾음
for tag in dom.select('form[action] input[name]'):
    print(tag.attrs['name'], tag.attrs['value'])
    # 얘네들이 모두 parameter들이다

In [ ]:
url = 'https://api.brunch.co.kr/v1/search/article'
params = {
    'q':'수지',
    'page': 1,  # page 넘기기 가능
    'pageSize': 20,
    'highlight':'y',    # n이면 이상하게 나옴
    'escape':'y',   # n이면 그대로 나옴
    'sortBy': 'accu'    # 최신식 객체를 해당으로 만들어놓음
}

resp = request('GET', url, params=params)

In [ ]:
resp.status_code, resp.reason, resp.headers, resp.request.headers

In [ ]:
resp.json().keys()

In [ ]:
for it in resp.json()['data']['list']:
    print(it['title'])

In [ ]:
url = 'https://brunch.co.kr/'
resp = request('GET', url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
dom.select('form')

In [ ]:
url = 'https://api.brunch.co.kr/v1/search/live'
params = {
    'q': '수지'
}
resp = request('GET', url, params=params)
resp.status_code, resp.headers['content-type']
resp.json()

In [ ]:
list(map(lambda r: r['title'], resp.json()['data']['article']))

In [ ]:
# 네이버 검색창- 자동완성 목록 가져오기
url = 'https://ac.search.naver.com/nx/ac'
params = {
    'q': '수지',
    'con': 0,
    'frm': 'nv',
    'ans': 2,
    'r_format': 'json',
    'r_enc':'UTF-8',
    'r_unicode': 0,
    't_koreng': 1,
    'run': 2,
    'rev': 4,
    'q_enc':'UTF-8',
    'st':100
}
# r_format 바꾸면 확장자 바뀔지도?
# _call_back의 있고 없고 차이가 있을 것임
# 즉, call_back은 여기서 다른 함수로 보내는 역할을 함
# 
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

resp = request('GET', url, params=params, headers={'user-agent':ua})
resp.status_code, resp.headers['content-type']

In [ ]:
list(map(lambda r: r[0], resp.json()['items'][0]))

In [ ]:
# 네이버 웹툰 수집
url = 'https://comic.naver.com/webtoon?tab=tue'
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
resp = request('GET', url, headers={'user-agent':ua})
resp.status_code, resp.headers['content-type']

In [ ]:
url = 'https://comic.naver.com/api/webtoon/titlelist/weekday' 
params = {
    'week':'tue',
    'order':'user'
}
resp = request('GET', url, params=params)
resp.status_code, resp.headers['content-type']

In [ ]:
list(map(lambda r: r['titleId'], resp.json()['titleList']))

In [ ]:
for node in resp.json()['titleList']:
    wurl = 'https://comic.naver.com/webtoon/list?titleId={}&tab=tue'.format(
        node['titleId']
    )
    count = 1
    new_url = 'https://comic.naver.com/api/article/list?titleId={}&page={}'.format(
        node['titleId'],
        count
    )
    resp = request('GET', new_url)
    num = resp.json()['articleList'][0]['no']
    too_url = 'https://comic.naver.com/webtoon/detail?titleId={}&no={}&week=tue'.format(
        node['titleId'],
        num
    )
    resp = request('GET', too_url)
    dom = BeautifulSoup(resp.text, 'html.parser')
    dom.select('#sectionContWide > img[src]')
# 이거 돌리면 웹툰 이미지 가져올 수 있음

In [178]:
# 웹툰 리스트 가져오는 함수
def webList(week):
    url = 'https://comic.naver.com/api/webtoon/titlelist/weekday' 
    params = {
        'week':'tue',
        'order':'user'
    }
    resp = request('GET', url, params=params)
    if resp.status_code == 200 and re.search(r'application\/json', resp.headers['content-type']):
        return resp.json()
    return None

# 웹툰 이름으로 찾기
def webNameSear(title,wl):
    for tag in filter(lambda r:r['titleName'] == title, wl['titleList']):
        return dict(tag['titleId'], tag['titleName'])

In [ ]:
# 웹툰 페이지 수로 가져오기
def webNo(tid, page=1):
    url = 'https://comic.naver.com/api/article/list?titleId={}&page={}'.format(
        tid,
        page
    )
    resp = request('GET', url)
    if resp.status_code == 200 and re.search(r'application\/json', resp.headers['content-type']):
        return resp.json()
    return None

In [ ]:
# 이미지 다운로더
def imgDown(tid, no):
    url = 'https://comic.naver.com/webtoon/detail?titleId={}&no={}'.format(
        tid,
        no
    )
    resp = request('GET', url)
    
    if resp.status_code == 200 and re.search(r'text.html',resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        return list(map(lambda r:urljoin(resp.request.url, r.attrs['src']), dom.select('#sectionContWide > img[src]')))
    return None

In [180]:
# -----------------------------------------------------
# -----------------------------------------------------
# -----------------------------------------------------
# -----------------------------------------------------
# -----------------------------------------------------
# -----------------------------------------------------
# -----------------------------------------------------
# -----------------------------------------------------

In [181]:
# login
url = 'https://pythonscraping.com/pages/login.html'
resp = request('GET', url)
resp.text

'<form method="post" action="login.php">\nUsername: <input type="text" name="username"><br>\nPassword: <input type="text" name="password"><br>\n<input type="submit" value="Submit">\n</form>'

In [182]:
# 결국 로그인이란, 필요한 정보를 기입해서 서버에 요청을 보내는 행동
# 보내는 method는 post로 보낼 것
# POST: HTTP.Request.Body를 통해서 보냄
# body를 사용하므로 formdata를 찾으면 됨

In [185]:
dom = BeautifulSoup(resp.text, 'html.parser')
for tag in dom.select('form input[name]'):
    print(tag['name'])
form = dom.select_one('form')
form.attrs['action'], form.attrs['method']

username
password


('login.php', 'post')

In [187]:
url = urljoin(url, form.attrs['action'])
params = {
    'username': 'who',
    'password': 'ppwwdd'
}
resp = request('POST', url, data=params)
resp.status_code, resp.request.body, resp.headers
# header를 봐야하는 이유는 == 쿠키 세팅을 봐야하기 때문 그리고 session 또한 확인 필요

(200,
 'username=who&password=ppwwdd',
 {'Server': 'nginx', 'Date': 'Tue, 19 Mar 2024 03:24:43 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '27015', 'Connection': 'keep-alive', 'X-Powered-By': 'PHP/7.4.33, PleskLin', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'},
 'text/html; charset=UTF-8')